In [84]:
# HR Employee Attrition Predictor
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import graphviz
import seaborn as sns
import sklearn.metrics as metrics
import plotly.graph_objs as go

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import label_binarize
from sklearn.compose import ColumnTransformer, make_column_transformer
from matplotlib.colors import ListedColormap
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

In [125]:
def encodeOutputVariable(y):
    labelencoder_Y_Origin = LabelEncoder()
    y = labelencoder_Y_Origin.fit_transform(y.astype(str))
    return y

def encodeCategoricalData(X, index):
    # encode categorical data
    labelencoder_X_Origin = LabelEncoder()
    X[:, index] = labelencoder_X_Origin.fit_transform(X[:, index].astype(str))
    return X    

def encodeHotEncoder(X, numberOfCategories):
    onehotencoder = OneHotEncoder(categorical_features = [numberOfCategories])
    X = onehotencoder.fit_transform(X.astype(str)).toarray()  
    X = X[:, 1:]
    return X

def minimumValues(train):
    return [0 if math.isnan(x) else x for x in train]

def minimumDelay(x):
    return 0 if np.isnan(x) or x < 0 else x

def outputPredictorResults(y_test, y_pred, title):
    # output results for Neural network Classification
    print("\nFor", title, "Classification")
    print("Accuracy Score of Prediction : ", metrics.accuracy_score(y_test, y_pred) * 100)
    print("\nConfusion Matrix")
    print(pd.crosstab(y_test.ravel(), y_pred.ravel(), rownames=['True'], colnames=['Predicted'], margins=True))
    print("\nClassification Report")
    print(metrics.classification_report(y_test, y_pred))
    print("Zero One Loss: ", metrics.zero_one_loss(y_test, y_pred))
#     print("Log Loss:      ", metrics.log_loss(y_test, y_pred))
#     print("ROC AUC Score: ", metrics.roc_auc_score(y_test, y_pred, average="micro"))

    
def graphROCCurve(y_test, y_pred):    
    fpr = {}
    tpr = {}
    roc_auc = {}
    
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = metrics.roc_curve(y_test.ravel(), y_pred.ravel())
    roc_auc["micro"] = metrics.auc(fpr["micro"], tpr["micro"])
    
    plt.figure()
    lw = 2
    plt.plot(fpr["micro"], tpr["micro"], color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc["micro"])
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()

def graphFeaturesImportant(rf_classifier, dataset):
    trace = go.Scatter(
        y = rf_classifier.feature_importances_, 
        x = dataset.columns.values, mode = "markers",
        marker = dict(
            sizemode = "diameter", sizeref=1, size=13, 
            color=rf_classifier.feature_importances_, colorscale="Portland",
            showscale=True
        ),
        text = dataset.columns.values
    )
    data = [trace]

    layout = go.Layout(
        autosize = True,
        title = "Random Forest Feature Importance",
        hovermode = "closest",
        xaxis = dict(
            ticklen=5, showgrid=True, zeroline=True, showline=True
        ),
        yaxis = dict(
            title="Feature Importance", showgrid=True, zeroline=True,
            ticklen=5, gridwidth=2
        ),
        showlegend=False
    )
    
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)
    
# developing the Multi Layer Perceptron Neural Network
def creatingNeuralNetworkPredictor(X_train, y_train, X_test, y_test, preprocess):
    print("\nNeural Network Classifier Section")
    print("---------------------------------")
    
    # initialize the Multi Layer Perceptron Neural Network 
    mlp_classifier = MLPClassifier(solver="adam", alpha=1e-5, max_iter=500,
                               hidden_layer_sizes=(13, 13, 13))
    
#     oversampler = SMOTE(random_state=0)
#     smote_X_train, smote_y_train = oversampler.fit_sample(X_train, y_train)
    
    # hook up the preprocess step with the classifier params and create the pipeline
    model = make_pipeline(preprocess, mlp_classifier)
    
    # fitting the Multi Layer Perceptron to the training set
    model.fit(X_train, y_train)
    
    # Predicting the Test set results
    mlp_y_pred = model.predict(X_test)
    
    # use the threshold of error to determine whether a prediction is valid
#     mlp_y_pred = (mlp_y_pred > 0.5)
    
    # making the confusion matrix
    cm = metrics.confusion_matrix(y_test.ravel(), mlp_y_pred.ravel())
    
    print("Training set Score: ", model.score(X_train, y_train))
    print("Testing set Score: ", model.score(X_test, y_test))    
    
    # output results
    outputPredictorResults(y_test, mlp_y_pred, "Neural Network")
#     graphROCCurve(y_test, mlp_y_pred)
    
# developing the Random Forest Classifier
def creatingRandomForestPredictor(X_train, y_train, X_test, y_test, preprocess):
    print("\nRandom Forest Classifier Section")
    print("---------------------------------")
    
    # initialize the Multi Layer Perceptron Neural Network 
    random_forest_classifier = RandomForestClassifier(**{'n_jobs': -1,
        'n_estimators': 800,
        'warm_start': True, 
        'max_features': 0.3,
        'max_depth': 9,
        'min_samples_leaf': 2,
        'max_features' : 'sqrt',
        'random_state' : 0,
        'verbose': 0
                                                      })
    
#     oversampler = SMOTE(random_state=0)
#     smote_X_train, smote_y_train = oversampler.fit_sample(X_train, y_train)
    
    # hook up the preprocess step with the classifier params and create the pipeline
    model = make_pipeline(preprocess, random_forest_classifier)
    
    # fitting Random Forest to the training set
    model.fit(X_train, y_train)
    
    # Predicting the Test set results
    rf_y_pred = model.predict(X_test)
    
    # use the threshold of error to determine whether a prediction is valid
#     rf_y_pred = (rf_y_pred > 0.5)
    
    # making the confusion matrix
    cm = metrics.confusion_matrix(y_test.ravel(), rf_y_pred.ravel())
    
    print("Training set Score: ", model.score(X_train, y_train))
    print("Testing set Score: ", model.score(X_test, y_test))    
    
    # output results
    outputPredictorResults(y_test, rf_y_pred, "Random Forest")
#     graphFeaturesImportant(model, dataset)

In [126]:
# importing the data
dataset = pd.read_csv("./data/employee_attrition.csv")
dataset.head()
dataset = dataset.drop(["YearsWithCurrManager"], axis=1)

In [127]:
# using pd to have dataframe drop column by name
# X = dataset.drop(["YearsWithCurrManager"], axis=1).values
# X = np.delete(X, [1], axis=1)
# y = dataset.iloc[:, 1].values

In [128]:
# using ColumnTransformer only approach
transformed_data = dataset.loc[:, dataset.columns != "Attrition"]
X = transformed_data.values
y = dataset.Attrition.values

numerical_features = transformed_data.dtypes == "int64"
categorical_features = ~numerical_features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

preprocess = make_column_transformer(
    (categorical_features, OneHotEncoder()),
    (numerical_features, make_pipeline(SimpleImputer(), StandardScaler()))
)
# classifier = MLPClassifier(solver="adam", alpha=1e-5, max_iter=500,
#                                hidden_layer_sizes=(13, 13, 13))

# random_forest_classifier = RandomForestClassifier(**{'n_jobs': -1,
#         'n_estimators': 800,
#         'warm_start': True, 
#         'max_features': 0.3,
#         'max_depth': 9,
#         'min_samples_leaf': 2,
#         'max_features' : 'sqrt',
#         'random_state' : 0,
#         'verbose': 0
#                                                       })
# model = make_pipeline(preprocess, random_forest_classifier)
# model.fit(Z_train, yz_train)
# print("score ", model.score(Z_test, yz_test))

/anaconda/lib/python3.6/site-packages/sklearn/compose/_column_transformer.py:751: DeprecationWarning:

`make_column_transformer` now expects (transformer, columns) as input tuples instead of (columns, transformer). This has been introduced in v0.20.1. `make_column_transformer` will stop accepting the deprecated (columns, transformer) order in v0.22.



In [129]:
# y = dataset.iloc[:, 1].values
# # loop over dataframe header and index values
# for i in enumerate(dataset.columns.values):
#     print(i)

In [130]:
# # encode categorical data - needs to be index because X is an array
# X = encodeCategoricalData(X, 1)
# X = encodeCategoricalData(X, 3)
# X = encodeCategoricalData(X, 6)
# X = encodeCategoricalData(X, 10)
# X = encodeCategoricalData(X, 14)
# X = encodeCategoricalData(X, 16)
# X = encodeCategoricalData(X, 20)
# X = encodeCategoricalData(X, 21)

# # remove dummy columns
# X = encodeHotEncoder(X, 4)
# print("Total", len(X[0]), "features")

In [131]:
# y = encodeOutputVariable(y)

In [132]:
# splitting the dataset into the training set and test set
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [133]:
# feature scaling 
# sc = StandardScaler(copy=True, with_mean=True, with_std=True)
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [134]:
# outputting data summary
print("Summary Info About the Dataset")
print("Does category contain null values?")
print(dataset.isnull().any(), "\n")
print("Said Yes to Attrition: ", y[(y == 1)].size)
print("Said No to Attrition:  ", y[(y == 0)].size)
print("Total responses:       ", y.size)

creatingNeuralNetworkPredictor(X_train, y_train, X_test, y_test, preprocess)
creatingRandomForestPredictor(X_train, y_train, X_test, y_test, preprocess)

Summary Info About the Dataset
Does category contain null values?
Age                         False
Attrition                   False
BusinessTravel              False
DailyRate                   False
Department                  False
DistanceFromHome            False
Education                   False
EducationField              False
EmployeeCount               False
EmployeeNumber              False
EnvironmentSatisfaction     False
Gender                      False
HourlyRate                  False
JobInvolvement              False
JobLevel                    False
JobRole                     False
JobSatisfaction             False
MaritalStatus               False
MonthlyIncome               False
MonthlyRate                 False
NumCompaniesWorked          False
Over18                      False
OverTime                    False
PercentSalaryHike           False
PerformanceRating           False
RelationshipSatisfaction    False
StandardHours               False
StockOptionLevel